In [1]:
!pip install transformers
!pip install faiss-cpu
!pip install sentence-transformers


In [2]:
from transformers import pipeline
from retriever import initialize_retriever, retrieve_top_chunks

# Initialize the question-answering pipeline
qa_pipeline = pipeline(
    "question-answering",
    model="henryk/bert-base-multilingual-cased-finetuned-polish-squad1",
    tokenizer="henryk/bert-base-multilingual-cased-finetuned-polish-squad1"
)

# Paths to FAISS index and metadata
faiss_path = "/content/130_len_faiss_index.bin"
metadata_path = "/content/130_len_metadata.json"

# Initialize the retriever
index, metadata, retriever_model = initialize_retriever(faiss_path, metadata_path)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/700 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/711M [00:00<?, ?B/s]

Some weights of the model checkpoint at henryk/bert-base-multilingual-cased-finetuned-polish-squad1 were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/40.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


FAISS index loaded successfully.
Metadata loaded successfully.
Retriever model initialized successfully.


In [3]:
def generate_answer(query, index, metadata, retriever_model, qa_pipeline, initial_top_k=50, final_top_k=10):
    """Generate an answer for the given query by retrieving context and using the QA pipeline."""
    # Retrieve relevant chunks
    relevant_chunks = retrieve_top_chunks(query, index, metadata, retriever_model, initial_top_k, final_top_k)

    # Combine retrieved texts for context in QA pipeline
    context = " ".join(chunk['text'] for chunk in relevant_chunks)

    # Generate the answer
    answer = qa_pipeline({'context': context, 'question': query})

    return answer['answer']

In [4]:
# Test questions
questions = [
    "Co jest najważniejsze dla pracowników?",
    "Dlaczego ważna jest odpowiedzialność w biznesie?"
]

# Generate and print answers
for question in questions:
    answer = generate_answer(question, index, metadata, retriever_model, qa_pipeline)
    print(f"Question: {question}")
    print(f"Answer: {answer}\n")

Query: co jest najważniejsze dla pracowników
Retrieved in 0.1029 seconds
Average similarity distance of results: 1.0523
Results count after filtering: 10

Question: Co jest najważniejsze dla pracowników?
Answer: zaangażowanie w pracy

Query: dlaczego ważna jest odpowiedzialność w biznesie
Retrieved in 0.1264 seconds
Average similarity distance of results: 1.1836
Results count after filtering: 10

Question: Dlaczego ważna jest odpowiedzialność w biznesie?
Answer: syndrom

